In [4]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F

In [6]:
d_model = 512
num_heads = 8
drop_prob = 0.1
batch_size = 30
max_sequence_length = 200
ffn_hidden = 2048
num_layers = 5

In [7]:
def scaled_dot_product(q, k, v, mask=None):
    # q,k,v = 30 x 8 x 200 x 64
    d_k = q.size()[-1] # 64
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)#30 x 8 x 200 x 200
    if mask is not None:
        scaled += mask # 30 x 8 x 200 x 200
    attention = F.softmax(scaled, dim=-1)# 30 x 8 x 200 x 200
    values = torch.matmul(attention, v) # 30 x 8 x 200 x 64
    return values, attention

In [17]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_head):
        super().__init__()
        self.d_model = d_model # 512 dimension
        self.num_head = num_head # 8
        self.head_dim = d_model // num_head # 64
        self.qkv_layer = nn.Linear(d_model, 3*d_model) # Feed forward layer for propogation 512 * 1536
        self.linear_layer = nn.Linear(d_model, d_model) # 512 * 512

    def forward(self, x, mask=None):
        batch_size, sequence_length, d_model = x.size() # 30 x 200 x 512
        print(f"x.size(): {x.size()}")
        qkv = self.qkv_layer(x) # 30 x 200 x 1536
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.reshape(batch_size, sequence_length, self.num_head, 3 * self.head_dim)# 30 x 200 x 8 x 192
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.permute(0, 2, 1, 3)# 30 x 8 x 200 x 192
        print(f"qkv.size(): {qkv.size()}")
        q, k, v = qkv.chunk(3, dim=-1)# each are 30 x 8 x 200 x 64
        print(f"q size: {q.size()}, k size: {k.size()}, v size: {v.size()}, ")
        values, attention = scaled_dot_product(q, k, v, mask)# attention: 30 x 8 x 200 x 200 value: 30 x 8 x 200 x 64
        print(f"values.size(): {values.size()}, attention.size:{ attention.size()} ")
        values = values.reshape(batch_size, sequence_length, self.num_head * self.head_dim)# 30 x 200 x 512
        print(f"values.size(): {values.size()}")
        out = self.linear_layer(values)
        print(f"out.size(): {out.size()}")
        return out

In [50]:
class LayerNormalization(nn.Module):
    def __init__(self, parameter_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameter_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameter_shape))
        self.beta =  nn.Parameter(torch.zeros(parameter_shape))

    def forward(self, inputs):
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        print(f"Mean ({mean.size()})")
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        print(f"Standard Deviation  ({std.size()})")
        y = (inputs - mean) / std
        print(f"y: {y.size()}")
        out = self.gamma * y  + self.beta
        print(f"self.gamma: {self.gamma.size()}, self.beta: {self.beta.size()}")
        print(f"out: {out.size()}")
        return out

In [51]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionWiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)# 512 x 2048
        self.linear2 = nn.Linear(hidden, d_model)# 2048 x 512
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):# 30 x 200 x 512
        x = self.linear1(x)# 30 x 200 x 2048
        x = self.relu(x)# 30 x 200 x 2048
        x = self.dropout(x)# 30 x 200 x 2048
        x = self.linear2(x)# 30 x 200 x 512
        return x

In [52]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_head, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_head=num_head)
        self.norm1 = LayerNormalization(parameter_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionWiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameter_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self ,x):
        residual_x = x # 30 x 200 x 512
        print("------- ATTENTION 1 ------")
        x = self.attention(x, mask=None)# 30 x 200 x 512
        print("------- DROPOUT 1 ------")
        x = self.dropout1(x)# 30 x 200 x 512
        print("------- ADD AND LAYER NORMALIZATION 1 ------")
        x = self.norm1(x + residual_x) # # 30 x 200 x 512
        residual_x = x # # 30 x 200 x 512
        print("------- ATTENTION 2 ------")
        x = self.ffn(x)# 30 x 200 x 512
        print("------- DROPOUT 2 ------")
        x = self.dropout2(x)# 30 x 200 x 512
        print("------- ADD AND LAYER NORMALIZATION 2 ------")
        x = self.norm2(x + residual_x)# 30 x 200 x 512
        return x

In [53]:
class Encoder(nn.Module):
    def __init__(self, d_model,ffn_hidden, num_head, drop_prob, num_layers):
        super().__init__()
        self.layers = nn.Sequential(*[EncoderLayer(d_model, ffn_hidden, num_head, drop_prob)
                                      for _ in range(num_layers)])
    def forward(self, x):
        x = self.layers(x)
        return x
encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)

In [54]:
x = torch.randn( (batch_size, max_sequence_length, d_model) ) # includes positional encoding
out = encoder(x)

------- ATTENTION 1 ------
x.size(): torch.Size([30, 200, 512])
qkv.size(): torch.Size([30, 200, 1536])
qkv.size(): torch.Size([30, 200, 8, 192])
qkv.size(): torch.Size([30, 8, 200, 192])
q size: torch.Size([30, 8, 200, 64]), k size: torch.Size([30, 8, 200, 64]), v size: torch.Size([30, 8, 200, 64]), 
values.size(): torch.Size([30, 8, 200, 64]), attention.size:torch.Size([30, 8, 200, 200]) 
values.size(): torch.Size([30, 200, 512])
out.size(): torch.Size([30, 200, 512])
------- DROPOUT 1 ------
------- ADD AND LAYER NORMALIZATION 1 ------
Mean (torch.Size([30, 200, 1]))
Standard Deviation  (torch.Size([30, 200, 1]))
y: torch.Size([30, 200, 512])
self.gamma: torch.Size([512]), self.beta: torch.Size([512])
out: torch.Size([30, 200, 512])
------- ATTENTION 2 ------
------- DROPOUT 2 ------
------- ADD AND LAYER NORMALIZATION 2 ------
Mean (torch.Size([30, 200, 1]))
Standard Deviation  (torch.Size([30, 200, 1]))
y: torch.Size([30, 200, 512])
self.gamma: torch.Size([512]), self.beta: torch.